In [1]:
from datetime import datetime, time, timedelta
import math
import statistics
import random
random.seed(datetime.now().timestamp())
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option("display.precision", 1)
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2

## Вхідні дані

In [4]:
columns = ["x0","x1","x2","y1","y2","y3","y4","y5","yj","y'j"]
data = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, -1, 1, 1, -1, -1, 1, -1, 1, -1],
    [-1, 1, -1, 1, -1, 1, 1, 1, -1, 1],
    [3, 9, 2, 5, 4, 4, 6, 4, 8, 1],
    [9, 2, 8, 3, 4, 6, 10, 3, 7, 9],
    [9, 1, 4, 3, 3, 1, 10, 4, 3, 7],
    [3, 10, 8, 1, 8, 7, 6, 2, 3, 9],
    [5, 2, 4, 2, 1, 1, 5, 2, 7, 10],
    [5.8, 4.8, 5.2, 2.8, 4, 3.8, 7.4, 3, 5.6, 7.2],
    [3.6, 5.6, 3.6, 6, 3.2, 5.6, 6, 5.6, 3.6, 5.6]
])
df = pd.DataFrame(np.transpose(data), columns=columns)
n = 5
N = 10
print(df)

    x0   x1   x2   y1    y2    y3    y4    y5   yj  y'j
0  1.0  1.0 -1.0  3.0   9.0   9.0   3.0   5.0  5.8  3.6
1  1.0 -1.0  1.0  9.0   2.0   1.0  10.0   2.0  4.8  5.6
2  1.0  1.0 -1.0  2.0   8.0   4.0   8.0   4.0  5.2  3.6
3  1.0  1.0  1.0  5.0   3.0   3.0   1.0   2.0  2.8  6.0
4  1.0 -1.0 -1.0  4.0   4.0   3.0   8.0   1.0  4.0  3.2
5  1.0 -1.0  1.0  4.0   6.0   1.0   7.0   1.0  3.8  5.6
6  1.0  1.0  1.0  6.0  10.0  10.0   6.0   5.0  7.4  6.0
7  1.0 -1.0  1.0  4.0   3.0   4.0   2.0   2.0  3.0  5.6
8  1.0  1.0 -1.0  8.0   7.0   3.0   3.0   7.0  5.6  3.6
9  1.0 -1.0  1.0  1.0   9.0   7.0   9.0  10.0  7.2  5.6


## Перевірити результати експериментальних досліджень:
#### –	на однорідність вибіркових дисперсій за критерієм Кохрена;

In [49]:
y_rows = data[3:8]
mean_square_error = [
    np.sum((y_row - y_row.mean())**2)/(len(y_row)-1)
    for y_row in y_rows]
G_criterion_Kohren = np.max(mean_square_error) / np.sum(mean_square_error)
f1 = n - 1
f2 = N
alpha = 0.05
print(G_criterion_Kohren)
#chi2.sf(q, f1)
#chi2.ppf(q, f1)

0.23567041965199592


In [50]:
G_table_Kohren_f1_f2_alpha = 0.3311
print(G_criterion_Kohren < G_table_Kohren_f1_f2_alpha)

True


G < Gkr, гіпотеза про однорідність дисперсій приймається

## Перевірити результати експериментальних досліджень:
#### –	на значущість коефіцієнтів регресії за критерієм Стьюдента;

In [51]:
yU = data[3]
x_rows = data[0:3]
bi = [(np.sum(yU * xiU)/N) for xiU in x_rows] ## regression coefficients
display(bi)

[4.6, 0.2, 1.2]

In [54]:
S2y = 1/(N*(n-1))*np.sum([np.sum([(yi - y_row.mean())**2 for yi in y_row]) for y_row in y_rows])
Sy = np.sqrt(S2y)
Sb = Sy/np.sqrt(n*N)
tp = np.absolute(bi)/Sb
display(tp)

array([10.40628107,  0.452447  ,  2.71468202])

In [55]:
f = N*(n-1)
alpha = 0.05

In [56]:
f

40

In [57]:
t_table_alpha_f = 2.021

Критерій є значущим:

In [58]:
significance = tp > t_table_alpha_f
display(significance)

array([ True, False,  True])

## Перевірити результати експериментальних досліджень:
#### –	на адекватність моделі за критерієм Фішера.

In [59]:
y_target = data[8]
## target or regression based on bi ???
significant_y_rows = []
for i in range(len(significance)):
    if significance[i]:
        significant_y_rows.append(y_rows[i])
l = len(significant_y_rows)
sigma2sum = 0
for i in range(N):
    scatter = 0
    experiment_count = len(significant_y_rows)
    for j in range(experiment_count):
        scatter += significant_y_rows[j][i]
    scatter /= experiment_count
    sigma2sum += (scatter - y_target[i])**2
S2x = n/(N-l)*sigma2sum
Fp = S2x / S2y
Fp

1.2736693961105428

In [60]:
alpha = 0.05
f1 = N - l
f1

8

In [61]:
f2 = N*(n-l)
f2

30

In [63]:
F_table_alpha_f1_f2 = 2.2662
Fp < F_table_alpha_f1_f2

True

Fp < F_table, гіпотеза про адекватність опису досліджуваного об'єкта отриманою математичною моделлю приймається